# Extended RNA-Seq Analysis Training Demo

## Overview

For simplicity and time, The short tutorial workflow uses truncated and partial run data from the Cushman et al., project.

The tutorial repeats the short tutorial, but with the full fastq files and includes some extra steps, such as how to download and prepare the transcriptome files used by salmon, alternate ways to navigate the NCBI databases for annotation or reference files you might need, and how to combine salmon outputs at the end into a single genecount file.

Full fastq files can be rather large, and so the downloading, extracting, and analysis of them means this tutorial can take over 1 hour 45 minutes to run the code fully. This is part of the reason we have a short and easy introductory tutorial, and this longer more full tutorial for those interested.

If this is too lengthy feel free to move on to the snakemake tutorial or the DEG analysis tutorial -- all the files used in the DEG tutorial were created using this extended tutorial workflow.

![RNA-Seq workflow](images/rnaseq-workflow.png)

## Learning Objectives

* **Install necessary bioinformatics tools:**  Learn to install and manage bioinformatics software using mamba.

* **Set up a project directory structure:** Organize files efficiently for RNA-Seq analysis.

* **Download RNA-Seq data from the SRA:** Utilize `prefetch` and `fasterq-dump` to download and convert SRA data to FASTQ format.  Learn to obtain accession numbers from NCBI databases (both manually and using BigQuery).

* **Quality control of raw reads:** Use `FastQC` to assess the quality of raw sequencing reads and `MultiQC` to generate a summary report.

* **Read trimming and adapter removal:** Employ `Trimmomatic` to improve read quality by removing adapter sequences and low-quality bases.

* **Transcriptome preparation:** Download and prepare a reference transcriptome using `entrez-direct` and `gffread`, including creating a decoy file for Salmon.

* **RNA-Seq read alignment and quantification:** Use `Salmon` to align reads to the transcriptome and quantify gene expression levels.

* **Gene expression analysis:**  Interpret Salmon output to identify highly expressed genes and analyze the expression of specific genes of interest.

* **Combine gene counts:** Merge individual sample quantification results into a single gene count table for downstream analysis (e.g., differential expression).

## Prerequisites

**APIs:**

* **gsutil:**  The Google Cloud Storage (GCS) tool `gsutil` is used extensively to download data from a Google Cloud Storage bucket.  This implicitly requires the Google Cloud Storage API to be enabled.

**Software and Dependencies:**

* **Trimmomatic:**  Used for quality trimming of raw sequencing reads.
* **FastQC:**  A quality control tool for high-throughput sequence data.
* **MultiQC:**  Aggregates results from multiple FastQC runs into a single report.
* **Salmon:** A tool for quantifying transcript abundances from RNA-Seq data.
* **Entrez Direct (EDirect):** NCBI's command-line tool for accessing the Entrez databases (used here to retrieve reference genome information).
* **gffread:**  Parses GFF/GTF annotation files and extracts information from them, such as to create a transcriptome reference file from genome and annotation files.
* **parallel-fastq-dump:**  A parallel version of the `fastq-dump` tool (part of SRA Toolkit).  It is likely optimized to process multiple files efficiently.
* **sra-tools:** NCBI's SRA Toolkit for downloading and processing data from the Sequence Read Archive.  Specifically, `prefetch` and `fasterq-dump` are used here.
* **pigz:** A parallel version of the gzip compression/decompression tool.  It's faster for larger files.
* **gsutil:** Google Cloud Storage (GCS) command line tool (used to download the accession list file).
* **BigQuery (optional):** Google's data warehouse service (used optionally to generate an accession list file, requires the BigQuery API to be enabled in your GCP Project and  proper authentication credentials in the environment).

### STEP 1: Install the tools

Using mamba and bioconda, install the tools that will be used in this tutorial.

In [1]:
!curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh 
!bash Miniforge3-$(uname)-$(uname -m).sh -b -u -p $HOME/miniforge 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 81.7M  100 81.7M    0     0   165M      0 --:--:-- --:--:-- --:--:--  165M
PREFIX=/home/jupyter/miniforge
Unpacking bootstrapper...
ln: failed to create symbolic link '/home/jupyter/miniforge/_conda': File exists


Show the system where to find the miniforge:

In [2]:
import os
os.environ["PATH"] += os.pathsep + os.environ["HOME"]+"/miniforge/bin" 

In [3]:
! mamba install -y -c conda-forge -c bioconda trimmomatic fastqc multiqc salmon entrez-direct gffread parallel-fastq-dump sra-tools=3.0.5 pigz

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache
bioconda/linux-64                                           Using cache
bioconda/noarch                                             Using cache
[+] 0.0s

Pinned packages:

  - python=3.10


Transaction

  Prefix: /opt/conda

  Updating specs:

   - trimmomatic
   - fastqc
   - multiqc
   - salmon
   - entrez-direct
   - gffread
   - parallel-fastq-dump
   - sra-tools=3.0.5
   - pigz


  Package                           Version  Build             Channel          Size
──────────────────────────────────────────────────────────────────────────────────────
  Install:
──────────────────────────────────────────────────────────────────────────────────────

  + curl                             8.17.0  h4e3cde8_0        conda-forge     186kB
  + entrez-direct                      24.0  he881be0_0        bioconda         17MB
  + gffread                

In [4]:
! prefetch --version


prefetch : 3.0.5



### STEP 2: Setup Environment

Create a set of directories to store the reads, reference sequence files, and output files. Notice that first we remove the `data` directory to clean up files from Tutorial_1


In [5]:
! cd $HOMEDIR
! echo $PWD
! rm -r data/
! mkdir -p data
! mkdir -p data/raw_fastq
! mkdir -p data/trimmed
! mkdir -p data/fastqc
! mkdir -p data/aligned
! mkdir -p data/reference

/home/jupyter/rnaseq-myco-notebook


Set number of cores depending on your VM size

In [6]:
numthreads=!nproc
numthreadsint = int(numthreads[0])
%env CORES = $numthreadsint
#!echo ${CORES}

env: CORES=4


### STEP 3: Downloading relevant FASTQ files using SRA Tools

Next we will need to download the relevant fastq files.

Because these files can be large, the process of downloading and extracting fastq files can be quite lengthy.

The sequence data for this tutorial comes from work by Cushman et al., <em><a href='https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8191103/'>Increased whiB7 expression and antibiotic resistance in Mycobacterium chelonae carrying two prophages</a><em>.

We will be downloading the sample runs from this project using SRA tools, downloading from the NCBI's SRA (Sequence Run Archives).

However, first we need to find the associated accession numbers in order to download.


### STEP 3.1: Finding run accession numbers.

The SRA stores sequence data in terms of runs, (SRR stands for Sequence Read Run). To download runs, we will need the accession ID for each run we wish to download. 

The Cushman et al., project contains 12 runs. To make it easier, these are the run IDs associated with this project:

+ SRR13349122
+ SRR13349123
+ SRR13349124
+ SRR13349125
+ SRR13349126
+ SRR13349127
+ SRR13349128
+ SRR13349129
+ SRR13349130
+ SRR13349131
+ SRR13349132
+ SRR13349133


In this case, all these runs belong to the SRP (Sequence Run Project): SRP300216.

Sequence run experiments can be searched for using the SRA database on the NCBI website; and article-specific sample run information can be found in the supplementary section of that article.

For instance, here, the the authors posted a link to the sequence data GSE (Gene Series number), <a href='https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE164210'>GSE164210</a>. This leads to the appropriate 'Gene Expression Omnibus' page where, among other useful files and information, the relevant SRA database link can be found. 

You can download this text file with the accession numbers and continue to STEP 3.2, or you can optionally use BigQuery to generate an accession list following the instructions outlined in [this notebook](https://github.com/STRIDES/NIHCloudLabGCP/blob/main/notebooks/SRADownload/SRA-Download.ipynb).
### STEP 3.1.1: Download the accession list file with `gsutil`

In [7]:
! gsutil cp gs://nigms-sandbox/me-inbre-rnaseq-pipelinev2/accs.txt .

Copying gs://nigms-sandbox/me-inbre-rnaseq-pipelinev2/accs.txt...
/ [1 files][  144.0 B/  144.0 B]                                                
Operation completed over 1 objects/144.0 B.                                      


### STEP 3.1.2 (Optional): Generate the accession list file with BigQuery

In [8]:
# Import the biquery api
from google.cloud import bigquery

Now make sure you have enabled the BigQuery API. You just need to search for BigQuery, go to the BQ page and click `Enable`

In [9]:
# Designate the client for the API
client = bigquery.Client(location="US")
print("Client creating using default project: {}".format(client.project))

Client creating using default project: kinglab-maine-strides


The table we are working with has the following Id:

In [10]:
table_id = "nih-sra-datastore.sra.metadata"

It has the following column names:

In [11]:
table_ref = client.get_table(table_id)
column_names = [field.name for field in table_ref.schema]
column_names

['acc',
 'assay_type',
 'center_name',
 'consent',
 'experiment',
 'sample_name',
 'instrument',
 'librarylayout',
 'libraryselection',
 'librarysource',
 'platform',
 'sample_acc',
 'biosample',
 'organism',
 'sra_study',
 'releasedate',
 'bioproject',
 'mbytes',
 'loaddate',
 'avgspotlen',
 'mbases',
 'insertsize',
 'library_name',
 'biosamplemodel_sam',
 'collection_date_sam',
 'geo_loc_name_country_calc',
 'geo_loc_name_country_continent_calc',
 'geo_loc_name_sam',
 'ena_first_public_run',
 'ena_last_update_run',
 'sample_name_sam',
 'datastore_filetype',
 'datastore_provider',
 'datastore_region',
 'attributes',
 'run_file_version',
 'jattr']

The first column (`acc`), is accession ID.

Now we will query BigQuery using the species name and a range of accession numbers associated with this particular study. Feel free to play around with the query to generate different variations of accession numbers!

In [12]:
query = """
#standardSQL
SELECT acc
FROM `nih-sra-datastore.sra.metadata`
WHERE organism = 'Mycobacteroides chelonae'
and acc LIKE '%SRR133491%'
ORDER BY acc
"""
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

In [13]:
result=list(query_job)
result

[Row(('SRR13349122',), {'acc': 0}),
 Row(('SRR13349123',), {'acc': 0}),
 Row(('SRR13349124',), {'acc': 0}),
 Row(('SRR13349125',), {'acc': 0}),
 Row(('SRR13349126',), {'acc': 0}),
 Row(('SRR13349127',), {'acc': 0}),
 Row(('SRR13349128',), {'acc': 0}),
 Row(('SRR13349129',), {'acc': 0}),
 Row(('SRR13349130',), {'acc': 0}),
 Row(('SRR13349131',), {'acc': 0}),
 Row(('SRR13349132',), {'acc': 0}),
 Row(('SRR13349133',), {'acc': 0})]

In [14]:
with open('accs2.txt', 'w') as f:
    for acc in result:
        f.write(acc[0]+'\n')

In [15]:
cat accs2.txt

SRR13349122
SRR13349123
SRR13349124
SRR13349125
SRR13349126
SRR13349127
SRR13349128
SRR13349129
SRR13349130
SRR13349131
SRR13349132
SRR13349133


### STEP 3.2: Using the SRA-toolkit for a single sample.

Now use the Sequence Run accession ID to download the sequence data.

In [16]:
! prefetch SRR13349124 -O data/raw_fastq -f yes


2026-01-05T18:12:58 prefetch.3.0.5: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
2026-01-05T18:12:58 prefetch.3.0.5: 1) Downloading 'SRR13349124'...
2026-01-05T18:12:58 prefetch.3.0.5: SRA Normalized Format file is being retrieved, if this is different from your preference, it may be due to current file availability.
2026-01-05T18:12:58 prefetch.3.0.5:  Downloading via HTTPS...
2026-01-05T18:13:18 prefetch.3.0.5:  HTTPS download succeed
2026-01-05T18:13:20 prefetch.3.0.5:  'SRR13349124' is valid
2026-01-05T18:13:20 prefetch.3.0.5: 1) 'SRR13349124' was downloaded successfully
2026-01-05T18:13:20 prefetch.3.0.5: 'SRR13349124' has 0 dependencies


Notice the SRA archives sequence files in the SRA format. 

Typically genome workflows process data in the form of zipped or unzipped .fastq, or .fasta files

So before we move on, we need to convert the files from .sra to .fastq using the fastq-dump tool.

We will also compresss the fastq files to make them take less space, making them fastq.gz files.

In [17]:
! for x in `cat accs.txt`; do fasterq-dump -f -O data/raw_fastq -e $CORES -m 4G data/raw_fastq/$x/$x.sra; done
# Note that you will get a result only for SRR13349124. Igonre error related to other accession IDs

2026-01-05T18:13:20 fasterq-dump.3.0.5 err: name not found while resolving query within virtual file system module - failed to resolve accession 'data/raw_fastq/SRR13349122/SRR13349122.sra' - Cannot resolve accession ( 404 )
fasterq-dump quit with error code 3
2026-01-05T18:13:20 fasterq-dump.3.0.5 err: name not found while resolving query within virtual file system module - failed to resolve accession 'data/raw_fastq/SRR13349123/SRR13349123.sra' - Cannot resolve accession ( 404 )
fasterq-dump quit with error code 3
spots read      : 10,727,273
reads read      : 21,454,546
reads written   : 21,454,546
2026-01-05T18:13:51 fasterq-dump.3.0.5 err: name not found while resolving query within virtual file system module - failed to resolve accession 'data/raw_fastq/SRR13349125/SRR13349125.sra' - Cannot resolve accession ( 404 )
fasterq-dump quit with error code 3
2026-01-05T18:13:51 fasterq-dump.3.0.5 err: name not found while resolving query within virtual file system module - failed to res

### STEP 3.3 Downloading multiple files using the SRA-toolkit.

One may, as in our case, wish to download multiple runs at once.

To aid in this, SRA-tools supports batch downloading.

We can download multiple SRA files using a single line of code by creating a list of the SRA IDs we wish to download, and inputting that into the prefetch command.

And then feed that list into the sra-toolkit prefetch command. Note, it may take some time to download all the fastq files.

In [18]:
! prefetch -O data/raw_fastq/ --option-file accs.txt


2026-01-05T18:13:52 prefetch.3.0.5: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
2026-01-05T18:13:52 prefetch.3.0.5: 1) Downloading 'SRR13349122'...
2026-01-05T18:13:52 prefetch.3.0.5: SRA Normalized Format file is being retrieved, if this is different from your preference, it may be due to current file availability.
2026-01-05T18:13:52 prefetch.3.0.5:  Downloading via HTTPS...
2026-01-05T18:14:12 prefetch.3.0.5:  HTTPS download succeed
2026-01-05T18:14:14 prefetch.3.0.5:  'SRR13349122' is valid
2026-01-05T18:14:14 prefetch.3.0.5: 1) 'SRR13349122' was downloaded successfully
2026-01-05T18:14:14 prefetch.3.0.5: 'SRR13349122' has 0 unresolved dependencies

2026-01-05T18:14:14 prefetch.3.0.5: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
2026-01-05T18:14:14 prefetch.3.0.5: 2) Downloading 'SRR13349123'...
2026-01-05T18:14:14 prefetch.3.0.5: SRA Normalized Format file is being retrieved, 

### STEP 3.3 Converting Multiple SRA files to Fastq

We used fasterq-dump before to convert SRA files to fastq. However, fasterq-dump does not have native batch compatibility. As before, we will use a loop to convert each file in our list. In this case, we are going to convert to fastq.gz for downstream processing. This step should take about 30 minutes.

In [19]:
! for x in `cat accs.txt`; do fasterq-dump -f -O data/raw_fastq -e $CORES -m 4G data/raw_fastq/$x/$x.sra; done

spots read      : 10,827,590
reads read      : 21,655,180
reads written   : 21,655,180
spots read      : 11,165,256
reads read      : 22,330,512
reads written   : 22,330,512
spots read      : 10,727,273
reads read      : 21,454,546
reads written   : 21,454,546
spots read      : 10,992,686
reads read      : 21,985,372
reads written   : 21,985,372
spots read      : 12,267,497
reads read      : 24,534,994
reads written   : 24,534,994
spots read      : 12,563,032
reads read      : 25,126,064
reads written   : 25,126,064
spots read      : 12,652,387
reads read      : 25,304,774
reads written   : 25,304,774
spots read      : 12,961,793
reads read      : 25,923,586
reads written   : 25,923,586
spots read      : 10,083,015
reads read      : 20,166,030
reads written   : 20,166,030
spots read      : 10,491,160
reads read      : 20,982,320
reads written   : 20,982,320
spots read      : 11,341,357
reads read      : 22,682,714
reads written   : 22,682,714
spots read      : 11,603,881
reads read    

Convert to fastq.gz

In [20]:
! time pigz data/raw_fastq/*.fastq


real	20m53.257s
user	77m19.848s
sys	2m20.395s


### STEP 4: Copy reference transcriptome files that will be used by Salmon using E-Direct

Salmon is a tool that aligns RNA-Seq reads to a transcriptome.

So we will need a transcriptome reference file.

To get one, we can search through the NCBI assembly database, find an assembly, and download transcriptome reference files from that assembly using FTP links.

For instance, we will use the <a href='https://www.ncbi.nlm.nih.gov/assembly/GCF_001632805.1'>ASM163280v1</a> refseq assembly, found by searching through the NCBI assembly database. The FTP links can be accessed through the website in various ways, one way is to click the 'FTP directory for RefSeq assembly' link, found under 'Access the data', section.

Alternatively, if one were inclined, one could take the less common route and perform this through the NCBI command line tool suite called 'Entrez Direct' (EDirect).

This is an intricate and complicated set of tools, with many ways to do any one thing.

Below is an example of using an eDirect search query with a refseq identifier to obtain the relevant FTP directory, and then using that to download desired reference files.

In [21]:
#parse for the ftp link and download the genome reference fasta file

! esearch -db assembly -query GCF_001632805.1 | efetch -format docsum \
| xtract -pattern DocumentSummary -element FtpPath_RefSeq \
| awk -F"/" '{print "curl -o data/reference/"$NF"_genomic.fna.gz " $0"/"$NF"_genomic.fna.gz"}' \
| bash

#parse for the ftp link and download the gtf reference fasta file

! esearch -db assembly -query GCF_001632805.1 | efetch -format docsum \
| xtract -pattern DocumentSummary -element FtpPath_RefSeq \
| awk -F"/" '{print "curl -o data/reference/"$NF"_genomic.gff.gz " $0"/"$NF"_genomic.gff.gz"}' \
| bash

# parse for the ftp link and download the feature-table reference file 
# (for later use for merging readcounts with gene names in R code).

! esearch -db assembly -query GCF_001632805.1 | efetch -format docsum \
| xtract -pattern DocumentSummary -element FtpPath_RefSeq \
| awk -F"/" '{print "curl -o data/reference/"$NF"_feature_table.txt.gz " $0"/"$NF"_feature_table.txt.gz"}' \
| bash


#unzip the compresseed fasta files

! gzip -d data/reference/*.gz --force

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1436k 100  1436k   0     0 10849k     0  --:--:-- --:--:-- --:--:-- 10886k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 426539 100 426539   0     0  1186k     0  --:--:-- --:--:-- --:--:--  1190k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 231931 100 231931   0     0  2609k     0  --:--:-- --:--:-- --:--:--  2633k


Next we can use a tool called gffread to create a transcriptome reference file using the gtf and genome files we downloaded.

In [22]:
! gffread -w data/reference/GCF_001632805.1_transcriptome_reference.fa -g data/reference/GCF_001632805.1_ASM163280v1_genomic.fna data/reference/GCF_001632805.1_ASM163280v1_genomic.gff

FASTA index file data/reference/GCF_001632805.1_ASM163280v1_genomic.fna.fai created.


It is also recommended to include the full genome at the end of the transcriptome reference file, for the purpose of performing a 'decoy-aware' mapping, more information about which can be found in the Salmon documentation.

To alert the tool to the presence of this, we will also create a 'decoy file', which salmon needs pointed towards the full genome sequence in our transcriptome reference file.

In [23]:
! cat data/reference/GCF_001632805.1_transcriptome_reference.fa <(echo) data/reference/GCF_001632805.1_ASM163280v1_genomic.fna > data/reference/GCF_001632805.1_transcriptome_reference_w_decoy.fa
! echo "NZ_CP007220.1" > data/reference/decoys.txt

### STEP 5: Copy data file for Trimmomatic

One of trimmomatics functions is to trim sequence machine specific adapter sequences. These are usually within the trimmomatic installation directory in a folder called adapters.

Directories of packages within mamba installations can be confusing, so in the case of using mamba with trimmomatic, it may be easier to simply download or create a file with the relevant adapter sequencecs and store it in an easy to find directory.

In [24]:
! gsutil -m cp -r gs://nigms-sandbox/me-inbre-rnaseq-pipelinev2/config/TruSeq3-PE.fa .
! head TruSeq3-PE.fa 

Copying gs://nigms-sandbox/me-inbre-rnaseq-pipelinev2/config/TruSeq3-PE.fa...
/ [1/1 files][   95.0 B/   95.0 B] 100% Done                                    
Operation completed over 1 objects/95.0 B.                                       
>PrefixPE/1
TACACTCTTTCCCTACACGACGCTCTTCCGATCT
>PrefixPE/2
GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT



### STEP 6: Run Trimmomatic
Trimmomatic will trim off any adapter sequences or low quality sequence it detects in the FASTQ files.

Using piping and our original list, it is possible to queue up a batch run of trimmomatic for all our files, note that this is a different way to run a loop compared with what we did before.

The below code may take approximately 35 minutes to run.

In [25]:
! cat accs.txt | xargs -I {} trimmomatic PE -threads $CORES 'data/raw_fastq/{}_1.fastq.gz' 'data/raw_fastq/{}_2.fastq.gz' 'data/trimmed/{}_1_trimmed.fastq.gz' 'data/trimmed/{}_1_trimmed_unpaired.fastq.gz' 'data/trimmed/{}_2_trimmed.fastq.gz' 'data/trimmed/{}_2_trimmed_unpaired.fastq.gz' ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36

TrimmomaticPE: Started with arguments:
 -threads 4 data/raw_fastq/SRR13349122_1.fastq.gz data/raw_fastq/SRR13349122_2.fastq.gz data/trimmed/SRR13349122_1_trimmed.fastq.gz data/trimmed/SRR13349122_1_trimmed_unpaired.fastq.gz data/trimmed/SRR13349122_2_trimmed.fastq.gz data/trimmed/SRR13349122_2_trimmed_unpaired.fastq.gz ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36
ILLUMINACLIP: Using adapter file from Trimmomatic installation folder: /opt/conda/share/trimmomatic-0.40-0/adapters/TruSeq3-PE.fa
Using PrefixPair: 'TACACTCTTTCCCTACACGACGCTCTTCCGATCT' and 'GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT'
ILLUMINACLIP: Using 1 prefix pairs, 0 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Read Pairs: 10827590 Both Surviving: 10810267 (99.84%) Forward Only Surviving: 17297 (0.16%) Reverse Only Surviving: 0 (0.00%) Dropped: 26 (0.00%)
TrimmomaticPE: Completed successfully
TrimmomaticPE: Started with

### STEP 7: Run FastQC
FastQC is an invaluable tool that allows you to evaluate whether there are problems with a set of reads. For example, it will provide a report of whether there is any bias in the sequence composition of the reads.

If you notice the results of the trimming, you may have noted the sequences in the reverse reads were few, and largely unpaired. This may be an artifact from how the original sequencing process. This is okay, we can proceed from here simply using the forward reads.

The below code may take around 10 minutes to run.

In [26]:
! cat accs.txt | xargs -I {} fastqc -t $CORES "data/trimmed/{}_1_trimmed.fastq.gz" -o data/fastqc/

application/gzip
Started analysis of SRR13349122_1_trimmed.fastq.gz
Approx 5% complete for SRR13349122_1_trimmed.fastq.gz
Approx 10% complete for SRR13349122_1_trimmed.fastq.gz
Approx 15% complete for SRR13349122_1_trimmed.fastq.gz
Approx 20% complete for SRR13349122_1_trimmed.fastq.gz
Approx 25% complete for SRR13349122_1_trimmed.fastq.gz
Approx 30% complete for SRR13349122_1_trimmed.fastq.gz
Approx 35% complete for SRR13349122_1_trimmed.fastq.gz
Approx 40% complete for SRR13349122_1_trimmed.fastq.gz
Approx 45% complete for SRR13349122_1_trimmed.fastq.gz
Approx 50% complete for SRR13349122_1_trimmed.fastq.gz
Approx 55% complete for SRR13349122_1_trimmed.fastq.gz
Approx 60% complete for SRR13349122_1_trimmed.fastq.gz
Approx 65% complete for SRR13349122_1_trimmed.fastq.gz
Approx 70% complete for SRR13349122_1_trimmed.fastq.gz
Approx 75% complete for SRR13349122_1_trimmed.fastq.gz
Approx 80% complete for SRR13349122_1_trimmed.fastq.gz
Approx 85% complete for SRR13349122_1_trimmed.fastq.g

### STEP 8: Run MultiQC
MultiQC reads in the FastQC reports and generate a compiled report for all the analyzed FASTQ files.

In [27]:
! multiqc -f data/fastqc/


/// ]8;id=929879;https://multiqc.info\MultiQC]8;;\ 🍾 v1.33

       file_search | Search path: /home/jupyter/rnaseq-myco-notebook/data/fastqc
         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 24/24  349123_1_trimmed_fastqc.html
            fastqc | Found 12 reports
     write_results | Data        : multiqc_data   (overwritten)
     write_results | Report      : multiqc_report.html   (overwritten)
           multiqc | MultiQC complete


### STEP 9: Index the Transcriptome so that Trimmed Reads Can Be Mapped Using Salmon

In [28]:
! salmon index -t data/reference/GCF_001632805.1_transcriptome_reference_w_decoy.fa -p $CORES -i data/reference/transcriptome_index --decoys data/reference/decoys.txt -k 31 --keepDuplicates

Version Server Response: Not Found
index ["data/reference/transcriptome_index"] did not previously exist  . . . creating it
[2026-01-05 19:29:24.154] [jLog] [info] building index
out : data/reference/transcriptome_index
[2026-01-05 19:29:24.155] [puff::index::jointLog] [info] Running fixFasta

[Step 1 of 4] : counting k-mers

[2026-01-05 19:29:24.422] [puff::index::jointLog] [warning] There were 1 transcripts that would need to be removed to avoid duplicates.
[2026-01-05 19:29:24.424] [puff::index::jointLog] [info] Replaced 0 non-ATCG nucleotides
[2026-01-05 19:29:24.424] [puff::index::jointLog] [info] Clipped poly-A tails from 0 transcripts
wrote 4930 cleaned references
[2026-01-05 19:29:24.444] [puff::index::jointLog] [info] Filter size not provided; estimating from number of distinct k-mers
[2026-01-05 19:29:24.548] [puff::index::jointLog] [info] ntHll estimated 4966944 distinct k-mers, setting filter size to 2^27
Threads = 4
Vertex length = 31
Hash functions = 5
Filter size = 13421

### STEP 10: Run Salmon to Map Reads to Transcripts and Quantify Expression Levels
Salmon aligns the trimmed reads to the reference transcriptome and generates the read counts per transcript. In this analysis, each gene has a single transcript.

In [29]:
! cat accs.txt | xargs -I {} salmon quant -i data/reference/transcriptome_index -l SR -r "data/trimmed/{}_1_trimmed.fastq.gz" -p $CORES --validateMappings -o "data/quants/{}_quant"

Version Server Response: Not Found
### salmon (selective-alignment-based) v1.10.3
### [ program ] => salmon 
### [ command ] => quant 
### [ index ] => { data/reference/transcriptome_index }
### [ libType ] => { SR }
### [ unmatedReads ] => { data/trimmed/SRR13349122_1_trimmed.fastq.gz }
### [ threads ] => { 4 }
### [ validateMappings ] => { }
### [ output ] => { data/quants/SRR13349122_quant }
Logs will be written to data/quants/SRR13349122_quant/logs
[2026-01-05 19:29:31.618] [jointLog] [info] setting maxHashResizeThreads to 4
[2026-01-05 19:29:31.618] [jointLog] [info] Fragment incompatibility prior below threshold.  Incompatible fragments will be ignored.
[2026-01-05 19:29:31.618] [jointLog] [info] Usage of --validateMappings implies use of minScoreFraction. Since not explicitly specified, it is being set to 0.65
[2026-01-05 19:29:31.618] [jointLog] [info] Setting consensusSlack to selective-alignment default of 0.35.
[2026-01-05 19:29:31.618] [jointLog] [info] parsing read library

### STEP 11: Report the top 10 most highly expressed genes in the samples

Top 10 most highly expressed genes in each wild-type sample.


In [30]:
! head data/quants/SRR13349122_quant/quant.sf -n 1
! sort -nrk 4,4 data/quants/SRR13349122_quant/quant.sf | head -10
! sort -nrk 4,4 data/quants/SRR13349123_quant/quant.sf | head -10
! sort -nrk 4,4 data/quants/SRR13349124_quant/quant.sf | head -10
! sort -nrk 4,4 data/quants/SRR13349125_quant/quant.sf | head -10
! sort -nrk 4,4 data/quants/SRR13349126_quant/quant.sf | head -10
! sort -nrk 4,4 data/quants/SRR13349127_quant/quant.sf | head -10

Name	Length	EffectiveLength	TPM	NumReads
rna-BB28_RS07080	117	3.947	366585.841449	16507.000
rna-BB28_RS07075	3114	2864.000	234607.889686	7665021.000
rna-BB28_RS07070	1516	1266.000	130436.454701	1883781.000
rna-BB28_RS17330	369	119.000	72495.625414	98414.000
gene-BB28_RS02220	204	9.377	8366.788570	895.000
gene-BB28_RS20695	231	14.032	6853.253254	1097.000
rna-BB28_RS09710	405	155.000	5461.510957	9657.000
gene-BB28_RS18745	300	51.326	3677.163924	2153.000
gene-BB28_RS18945	222	12.150	3527.969159	489.000
gene-BB28_RS24750	102	3.543	3389.529139	137.000
sort: write failed: 'standard output': Broken pipe
sort: write error
rna-BB28_RS07080	117	3.947	348187.981351	16529.000
rna-BB28_RS07075	3114	2864.000	224080.029008	7718168.000
rna-BB28_RS07070	1516	1266.000	132514.469884	2017600.000
rna-BB28_RS17330	369	119.000	76258.206854	109137.000
gene-BB28_RS02220	204	9.377	8716.636540	983.000
gene-BB28_RS20695	231	14.032	7306.558915	1233.000
rna-BB28_RS09710	405	155.000	6220.151911	11595.000
gene-BB28_R

Top 10 most highly expressed genes in the double lysogen samples.


In [ ]:
! head data/quants/SRR13349122_quant/quant.sf -n 1
! sort -nrk 4,4 data/quants/SRR13349128_quant/quant.sf | head -10
! sort -nrk 4,4 data/quants/SRR13349129_quant/quant.sf | head -10
! sort -nrk 4,4 data/quants/SRR13349130_quant/quant.sf | head -10
! sort -nrk 4,4 data/quants/SRR13349131_quant/quant.sf | head -10
! sort -nrk 4,4 data/quants/SRR13349132_quant/quant.sf | head -10
! sort -nrk 4,4 data/quants/SRR13349133_quant/quant.sf | head -10

Name	Length	EffectiveLength	TPM	NumReads
rna-BB28_RS07080	117	3.947	453587.771883	24652.000
rna-BB28_RS07075	3114	2864.000	246618.398908	9725111.000
rna-BB28_RS07070	1516	1266.000	92386.166160	1610411.000
rna-BB28_RS17330	369	119.000	77504.350847	126990.000
gene-BB28_RS20695	231	14.032	4922.347990	951.000
rna-BB28_RS09710	405	155.000	4304.730201	9187.000
gene-BB28_RS02220	204	9.377	3787.444358	489.000
gene-BB28_RS14885	195	8.348	2601.410265	299.000
gene-BB28_RS06975	216	11.099	2538.997624	388.000
gene-BB28_RS21780	213	10.625	2221.665444	325.000
sort: write failed: 'standard output': Broken pipe
sort: write error
rna-BB28_RS07080	117	3.947	436872.913506	24834.000
rna-BB28_RS07075	3114	2864.000	237194.551939	9783056.000
rna-BB28_RS07070	1516	1266.000	95484.488841	1740858.000
rna-BB28_RS17330	369	119.000	82989.979689	142223.000
gene-BB28_RS20695	231	14.032	5394.082164	1090.000
rna-BB28_RS09710	405	155.000	5010.356664	11184.000
gene-BB28_RS02220	204	9.377	4117.289994	556.000
gene-BB28_RS14

### STEP 12: Report the expression of a putative acyl-ACP desaturase (BB28_RS16545) that was downregulated in the double lysogen relative to wild-type
A acyl-transferase was reported to be downregulated in the double lysogen as shown in the table of the top 20 upregulated and downregulated genes from the paper describing the study.
![RNA-Seq workflow](images/table-cushman.png)

Use `grep` to report the expression in the wild-type sample. The fields in the Salmon `quant.sf` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [35]:
! grep 'BB28_RS16545' data/quants/SRR13349122_quant/quant.sf
! grep 'BB28_RS16545' data/quants/SRR13349123_quant/quant.sf
! grep 'BB28_RS16545' data/quants/SRR13349124_quant/quant.sf
! grep 'BB28_RS16545' data/quants/SRR13349125_quant/quant.sf
! grep 'BB28_RS16545' data/quants/SRR13349126_quant/quant.sf
! grep 'BB28_RS16545' data/quants/SRR13349127_quant/quant.sf

gene-BB28_RS16545	987	737.000	53.285962	448.000
gene-BB28_RS16545	987	737.000	50.205866	445.000
gene-BB28_RS16545	987	737.000	42.456975	403.000
gene-BB28_RS16545	987	737.000	44.542229	441.000
gene-BB28_RS16545	987	737.000	52.537906	452.000
gene-BB28_RS16545	987	737.000	53.865093	487.000


Use `grep` to report the expression in the double lysogen sample. The fields in the Salmon `quant.sf` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [36]:
! grep 'BB28_RS16545' data/quants/SRR13349128_quant/quant.sf
! grep 'BB28_RS16545' data/quants/SRR13349129_quant/quant.sf
! grep 'BB28_RS16545' data/quants/SRR13349130_quant/quant.sf
! grep 'BB28_RS16545' data/quants/SRR13349131_quant/quant.sf
! grep 'BB28_RS16545' data/quants/SRR13349132_quant/quant.sf
! grep 'BB28_RS16545' data/quants/SRR13349133_quant/quant.sf

gene-BB28_RS16545	987	737.000	4.631635	47.000
gene-BB28_RS16545	987	737.000	6.501069	69.000
gene-BB28_RS16545	987	737.000	5.088172	37.000
gene-BB28_RS16545	987	737.000	5.899337	45.000
gene-BB28_RS16545	987	737.000	4.929872	43.000
gene-BB28_RS16545	987	737.000	6.210249	57.000


### STEP 12: Combine Genecounts to a Single Genecount File
Commonly, the readcounts for each sample are combined into a single table, where the rows contain the gene ID, and the columns identify the sample.

In [37]:
##first merge salmon files by number of reads.
! salmon quantmerge --column numreads --quants data/quants/*_quant -o data/quants/merged_quants.txt
##optinally we can rename the columns
! sed -i "1s/.*/Name\tSRR13349122\tSRR13349123\tSRR13349124\tSRR13349125\tSRR13349126\tSRR13349127\tSRR13349128\tSRR13349129\tSRR13349130\tSRR13349131\tSRR13349132\tSRR13349133/" data/quants/merged_quants.txt

##for further formatting, it may be easier in our r-code to later merge
##if we remove the gene- and rna- prefix
! sed -i "s/gene-//" data/quants/merged_quants.txt
! sed -i "s/rna-//" data/quants/merged_quants.txt

print("An example of a combined genecount outputfile.")
! head data/quants/merged_quants.txt

Version Server Response: Not Found
[2026-01-05 19:38:49.936] [mergeLog] [info] samples: [ data/quants/SRR13349122_quant, data/quants/SRR13349123_quant, data/quants/SRR13349124_quant, data/quants/SRR13349125_quant, data/quants/SRR13349126_quant, data/quants/SRR13349127_quant, data/quants/SRR13349128_quant, data/quants/SRR13349129_quant, data/quants/SRR13349130_quant, data/quants/SRR13349131_quant, data/quants/SRR13349132_quant, data/quants/SRR13349133_quant ]
[2026-01-05 19:38:49.936] [mergeLog] [info] sample names : [ SRR13349122_quant, SRR13349123_quant, SRR13349124_quant, SRR13349125_quant, SRR13349126_quant, SRR13349127_quant, SRR13349128_quant, SRR13349129_quant, SRR13349130_quant, SRR13349131_quant, SRR13349132_quant, SRR13349133_quant ]
[2026-01-05 19:38:49.936] [mergeLog] [info] output column : NUMREADS
[2026-01-05 19:38:49.936] [mergeLog] [info] output file : data/quants/merged_quants.txt
[2026-01-05 19:38:49.936] [mergeLog] [info] Parsing data/quants/SRR13349122_quant/quant.sf

## <a name="workflow">Additional Workflows</a>

Now that you have read counts per gene, feel free to explore the R workflow which creates plots and analyses using these readcount files, or try other alternate workflows for creating read count files, such as using snakemake.


[Workflow One:](Tutorial_1.ipynb) A short introduction to downloading and mapping sequences to a transcriptome using Trimmomatic and Salmon. Here is a link to the YouTube video demonstrating the tutorial: <https://youtu.be/ChGfBR4do_Y>.

[Workflow One (Extended):](Tutorial_1B_Extended.ipynb) An extended version of workflow one. Once you have got your feet wet, you can retry workflow one with this extended version that covers the entire dataset, and includes elaboration such as using SRA tools for sequence downloading, and examples of running batches of fastq files through the pipeline. This workflow may take around an hour to run.

[Workflow One (Using Snakemake):](Tutorial_2_Snakemake.ipynb) Using snakemake to run workflow one.

[Workflow Two (DEG Analysis):](Tutorial_3_DEG_Analysis.ipynb) Using Deseq2 and R to conduct clustering and differential gene expression analysis.


![RNA-Seq workflow](images/RNA-Seq_Notebook_Homepage.png)

## Conclusion

This extended RNA-Seq analysis tutorial provided a comprehensive workflow for processing RNA-Seq data, from raw SRA files to a combined gene count table suitable for differential gene expression analysis.  We demonstrated the use of various bioinformatics tools, including `prefetch`, `fasterq-dump`, `Trimmomatic`, `FastQC`, `MultiQC`, `Salmon`, `entrez-direct`, and `gffread`,  highlighting best practices for data management and efficient command-line execution, particularly for batch processing of multiple samples. The tutorial included detailed steps for downloading data from the NCBI SRA database, utilizing both manual accession identification and an optional approach leveraging Google Cloud BigQuery for programmatic retrieval.  The integration of these tools facilitated quality control, read trimming, transcriptome preparation, read alignment, and quantification, culminating in a consolidated gene count file.  This workflow, although time-consuming due to the processing of full FASTQ datasets, provides a robust foundation for more advanced analyses such as those detailed in the subsequent Snakemake and DEG analysis tutorials.  The resulting gene count matrix serves as a crucial input for downstream differential expression analyses, building upon the knowledge gained in this comprehensive guide.

## Clean Up

Remember to move to the next notebook or shut down your instance if you are finished.